# Basic dynamic model

In [1]:
DATA_PATH = '../backend/data/dynamic_dataset'


In [2]:
#Load DATA
from sign.training.landmark_extraction.MediaPiper import MediaPiper

mp = MediaPiper()
gestures = mp.process_dynamic_gestures_from_folder(DATA_PATH)

2024-03-11 13:13:21.960425: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 13:13:21.962494: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 13:13:21.991336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 13:13:21.991364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 13:13:21.992164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

['2_1.jpg', '2_2.jpg', '2_3.jpg', '2_4.jpg', '2_5.jpg', '2_6.jpg', '2_7.jpg', '2_8.jpg', '2_9.jpg', '2_10.jpg', '2_11.jpg', '2_12.jpg', '2_13.jpg', '2_14.jpg', '6_1.jpg', '6_2.jpg', '6_3.jpg', '6_4.jpg', '6_5.jpg', '6_6.jpg', '6_7.jpg', '6_8.jpg', '6_9.jpg', '6_10.jpg', '6_11.jpg', '6_12.jpg', '6_13.jpg', '6_14.jpg', '10_1.jpg', '10_2.jpg', '10_3.jpg', '10_4.jpg', '10_5.jpg', '10_6.jpg', '10_7.jpg', '10_8.jpg', '10_9.jpg', '10_10.jpg', '10_11.jpg', '10_12.jpg', '10_13.jpg', '10_14.jpg']
['3_1.jpg', '3_2.jpg', '3_3.jpg', '3_4.jpg', '3_5.jpg', '3_6.jpg', '3_7.jpg', '3_8.jpg', '3_9.jpg', '3_10.jpg', '3_11.jpg', '3_12.jpg', '3_13.jpg', '3_14.jpg', '6_1.jpg', '6_2.jpg', '6_3.jpg', '6_4.jpg', '6_5.jpg', '6_6.jpg', '6_7.jpg', '6_8.jpg', '6_9.jpg', '6_10.jpg', '6_11.jpg', '6_12.jpg', '6_13.jpg', '6_14.jpg', '8_1.jpg', '8_2.jpg', '8_3.jpg', '8_4.jpg', '8_5.jpg', '8_6.jpg', '8_7.jpg', '8_8.jpg', '8_9.jpg', '8_10.jpg', '8_11.jpg', '8_12.jpg', '8_13.jpg', '8_14.jpg']


In [3]:
[gesture.label for gesture in gestures]

['fist-up', 'fist-down']

In [6]:
import numpy as np
from sign.trajectory import TrajectoryBuilder, trajectory

bob = TrajectoryBuilder()

gesture_trajector_map: dict[str, list[trajectory]] = {}

for gesture in gestures:
    gesture_trajector_map[gesture.label] = []
    for sequence in gesture.results:
        new_sequence = []
        for image_mp_res in sequence:
            hand_landmarks = np.array(image_mp_res.multi_hand_landmarks)
            new_sequence.append(hand_landmarks)
        sequence_as_np_array = np.array(new_sequence)
        sequence_trajectory = bob.make_trajectory(sequence_as_np_array)

        gesture_trajector_map[gesture.label].append(sequence_trajectory)

print(gesture_trajector_map)

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
{'fist-up': [trajectory(directions=[trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.UP: 1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.DOWN: -1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.UP: 1>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_element(x=<direction.STATIONARY: 0>, y=<direction.DOWN: -1>, z=<direction.STATIONARY: 0>), trajectory_e

In [23]:
tmp = [(k, [v2.directions for v2 in v]) for (k,v) in gesture_trajector_map.items()]
X = [tjs for _,tjs in tmp]
y = [label for label,_ in tmp]

for gesture in X:
    min = len(gesture[0])
    for trajectory in gesture[1:]:
        length = len(trajectory)
        if min > length:
            min = length
    for trajectory in gesture:
        if len(trajectory) > min:
            for i in range(len(trajectory) - min):
                trajectory.pop()

In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X,y)

ValueError: Found array with dim 3. LinearRegression expected <= 2.